# Feature Engineering for QuickBooks Sales Forecasting

This notebook focuses on transforming raw sales data into model-ready features for our forecasting model.


In [39]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set visualization style
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")
%matplotlib inline

In [40]:
# Load the sales data
sales_df = pd.read_csv('../data/raw/sales.csv')

# Convert date to datetime
sales_df['date'] = pd.to_datetime(sales_df['date'])

# Display the first few rows
sales_df.head()

,date,category,product,units_sold,revenue,product_price,promo_flag,is_holiday_season,category_popularity_30d
0,2022-05-09,Electronics,Phone,22,3699.08,168.14,0,0,22.00
1,2022-05-09,Electronics,Laptop,42,6213.48,147.94,0,0,32.00
2,2022-05-09,Electronics,Tablet,37,3061.38,82.74,1,0,33.67
3,2022-05-09,Groceries,Milk,4,71.96,17.99,0,0,4.00
4,2022-05-09,Groceries,Bread,45,8412.75,186.95,0,0,24.50


## Time-Based Features
In this section, we extract temporal features from the date column. These features capture important calendar-based patterns like yearly seasonality, monthly cycles, and day-of-week effects that are crucial for time series forecasting. We create features for year, month, day of week, and a binary weekend indicator.


In [41]:
# Extract date components
sales_df['year'] = sales_df['date'].dt.year
sales_df['month'] = sales_df['date'].dt.month
sales_df['day_of_week'] = sales_df['date'].dt.dayofweek
sales_df['is_weekend'] = sales_df['day_of_week'].isin([5, 6]).astype(int)

# Display the enhanced dataframe
sales_df.head()

,date,category,product,units_sold,revenue,product_price,promo_flag,is_holiday_season,category_popularity_30d,year,month,day_of_week,is_weekend
0,2022-05-09,Electronics,Phone,22,3699.08,168.14,0,0,22.00,2022,5,0,0
1,2022-05-09,Electronics,Laptop,42,6213.48,147.94,0,0,32.00,2022,5,0,0
2,2022-05-09,Electronics,Tablet,37,3061.38,82.74,1,0,33.67,2022,5,0,0
3,2022-05-09,Groceries,Milk,4,71.96,17.99,0,0,4.00,2022,5,0,0
4,2022-05-09,Groceries,Bread,45,8412.75,186.95,0,0,24.50,2022,5,0,0


## Lag and Rolling Features
In this section, we create lag and rolling window features that capture temporal dependencies in the data. Lag features represent past values (previous day, week, etc.), which help the model learn from historical patterns. Rolling window features (like 7-day and 30-day averages) smooth out short-term fluctuations and highlight longer-term trends. These features are essential for time series forecasting as they provide the model with historical context.


In [42]:
# Create lagged features (previous day, week, month)
sales_df = sales_df.sort_values(by=['product', 'date'])

# Lag
sales_df['lag_units_1d'] = sales_df.groupby('product')['units_sold'].shift(1)
sales_df['lag_revenue_1d'] = sales_df.groupby('product')['revenue'].shift(1)

# Rolling mean
sales_df['rolling_avg_units_7d'] = sales_df.groupby('product')['units_sold'].transform(lambda x: x.shift(1).rolling(7).mean())
sales_df['rolling_avg_units_30d'] = sales_df.groupby('product')['units_sold'].transform(lambda x: x.shift(1).rolling(30).mean())

# Display with lag features
sales_df.head(10)

,date,category,product,units_sold,revenue,product_price,promo_flag,is_holiday_season,category_popularity_30d,year,month,day_of_week,is_weekend,lag_units_1d,lag_revenue_1d,rolling_avg_units_7d,rolling_avg_units_30d
20,2022-05-09,Sports,Basketball,53,7204.29,135.93,1,0,38.00,2022,5,0,0,NaN,NaN,NaN,NaN
44,2022-05-10,Sports,Basketball,47,7289.23,155.09,0,0,33.67,2022,5,1,0,53.0,7204.29,NaN,NaN
68,2022-05-11,Sports,Basketball,7,1322.16,188.88,0,0,32.89,2022,5,2,0,47.0,7289.23,NaN,NaN
92,2022-05-12,Sports,Basketball,14,1763.44,125.96,0,0,31.33,2022,5,3,0,7.0,1322.16,NaN,NaN
116,2022-05-13,Sports,Basketball,23,2726.65,118.55,0,0,30.47,2022,5,4,0,14.0,1763.44,NaN,NaN
140,2022-05-14,Sports,Basketball,41,7183.20,175.20,0,0,30.83,2022,5,5,1,23.0,2726.65,NaN,NaN
164,2022-05-15,Sports,Basketball,30,3654.00,121.80,0,0,28.52,2022,5,6,1,41.0,7183.20,NaN,NaN
188,2022-05-16,Sports,Basketball,25,593.25,23.73,0,0,28.79,2022,5,0,0,30.0,3654.00,30.714286,NaN
212,2022-05-17,Sports,Basketball,7,821.59,117.37,0,0,27.48,2022,5,1,0,25.0,593.25,26.714286,NaN
236,2022-05-18,Sports,Basketball,28,4919.32,175.69,0,0,25.93,2022,5,2,0,7.0,821.59,21.000000,NaN


## Promotional and Price Features
In this section, we engineer features related to pricing strategies and promotional activities. We create a price change flag to identify when product prices change, which can significantly impact sales. We also create an interaction feature between promotions and holiday seasons, as the combined effect of these factors often leads to sales spikes. These features help the model understand how pricing and promotional strategies influence purchasing behavior.


In [43]:
# Price change from the previous day
sales_df['price_change_flag'] = sales_df.groupby('product')['product_price'].diff().fillna(0).ne(0).astype(int)

# Promo and holiday interaction
sales_df['promo_and_holiday'] = sales_df['promo_flag'] & sales_df['is_holiday_season']

sales_df.head(10)

,date,category,product,units_sold,revenue,product_price,promo_flag,is_holiday_season,category_popularity_30d,year,month,day_of_week,is_weekend,lag_units_1d,lag_revenue_1d,rolling_avg_units_7d,rolling_avg_units_30d,price_change_flag,promo_and_holiday
20,2022-05-09,Sports,Basketball,53,7204.29,135.93,1,0,38.00,2022,5,0,0,NaN,NaN,NaN,NaN,0,0
44,2022-05-10,Sports,Basketball,47,7289.23,155.09,0,0,33.67,2022,5,1,0,53.0,7204.29,NaN,NaN,1,0
68,2022-05-11,Sports,Basketball,7,1322.16,188.88,0,0,32.89,2022,5,2,0,47.0,7289.23,NaN,NaN,1,0
92,2022-05-12,Sports,Basketball,14,1763.44,125.96,0,0,31.33,2022,5,3,0,7.0,1322.16,NaN,NaN,1,0
116,2022-05-13,Sports,Basketball,23,2726.65,118.55,0,0,30.47,2022,5,4,0,14.0,1763.44,NaN,NaN,1,0
140,2022-05-14,Sports,Basketball,41,7183.20,175.20,0,0,30.83,2022,5,5,1,23.0,2726.65,NaN,NaN,1,0
164,2022-05-15,Sports,Basketball,30,3654.00,121.80,0,0,28.52,2022,5,6,1,41.0,7183.20,NaN,NaN,1,0
188,2022-05-16,Sports,Basketball,25,593.25,23.73,0,0,28.79,2022,5,0,0,30.0,3654.00,30.714286,NaN,1,0
212,2022-05-17,Sports,Basketball,7,821.59,117.37,0,0,27.48,2022,5,1,0,25.0,593.25,26.714286,NaN,1,0
236,2022-05-18,Sports,Basketball,28,4919.32,175.69,0,0,25.93,2022,5,2,0,7.0,821.59,21.000000,NaN,1,0


## Volatility Features
In this section, we create features that capture the volatility or variability in sales over time. We calculate the rolling standard deviation of units sold over a 7-day window, which helps identify periods of stable versus unstable sales. Volatility features are important for forecasting as they help the model adjust its predictions based on the historical stability of sales, potentially increasing confidence during stable periods and widening prediction intervals during volatile periods.


In [44]:
sales_df['rolling_std_units_7d'] = sales_df.groupby('product')['units_sold'].transform(lambda x: x.shift(1).rolling(7).std())
sales_df.head(10)

,date,category,product,units_sold,revenue,product_price,promo_flag,is_holiday_season,category_popularity_30d,year,month,day_of_week,is_weekend,lag_units_1d,lag_revenue_1d,rolling_avg_units_7d,rolling_avg_units_30d,price_change_flag,promo_and_holiday,rolling_std_units_7d
20,2022-05-09,Sports,Basketball,53,7204.29,135.93,1,0,38.00,2022,5,0,0,NaN,NaN,NaN,NaN,0,0,NaN
44,2022-05-10,Sports,Basketball,47,7289.23,155.09,0,0,33.67,2022,5,1,0,53.0,7204.29,NaN,NaN,1,0,NaN
68,2022-05-11,Sports,Basketball,7,1322.16,188.88,0,0,32.89,2022,5,2,0,47.0,7289.23,NaN,NaN,1,0,NaN
92,2022-05-12,Sports,Basketball,14,1763.44,125.96,0,0,31.33,2022,5,3,0,7.0,1322.16,NaN,NaN,1,0,NaN
116,2022-05-13,Sports,Basketball,23,2726.65,118.55,0,0,30.47,2022,5,4,0,14.0,1763.44,NaN,NaN,1,0,NaN
140,2022-05-14,Sports,Basketball,41,7183.20,175.20,0,0,30.83,2022,5,5,1,23.0,2726.65,NaN,NaN,1,0,NaN
164,2022-05-15,Sports,Basketball,30,3654.00,121.80,0,0,28.52,2022,5,6,1,41.0,7183.20,NaN,NaN,1,0,NaN
188,2022-05-16,Sports,Basketball,25,593.25,23.73,0,0,28.79,2022,5,0,0,30.0,3654.00,30.714286,NaN,1,0,17.172791
212,2022-05-17,Sports,Basketball,7,821.59,117.37,0,0,27.48,2022,5,1,0,25.0,593.25,26.714286,NaN,1,0,14.103360
236,2022-05-18,Sports,Basketball,28,4919.32,175.69,0,0,25.93,2022,5,2,0,7.0,821.59,21.000000,NaN,1,0,12.529964


## Daily Contextual Features
In this section, we create aggregated features at the daily level to provide broader context for each transaction. We calculate total sales, transaction count, and the number of unique categories sold each day. These daily contextual features help the model understand the overall business environment on a given day, which can be important for accurate forecasting. For example, a high transaction count day might indicate a sale event or holiday shopping period.


In [45]:
# Aggregate daily stats
daily_context = sales_df.groupby('date').agg(
    total_sales=('revenue', 'sum'),
    transaction_count=('revenue', 'count'),
    unique_categories=('category', 'nunique')
).reset_index()

# Merge to main df
sales_df = pd.merge(sales_df, daily_context, on='date', how='left')

## Category-Based Features
In this section, we create features that capture sales patterns at the category level. We aggregate sales by date and category, then pivot the data to create separate columns for each product category. This transformation allows the model to learn category-specific patterns and relationships. Understanding how different product categories perform over time is crucial for accurate forecasting, especially when certain categories have distinct seasonal patterns or growth trends.


In [46]:
# Create category-specific features
category_daily = sales_df.groupby(['date', 'category'])['revenue'].sum().reset_index()

# Pivot to get categories as columns
category_pivot = category_daily.pivot(index='date', columns='category', values='revenue').reset_index()
category_pivot = category_pivot.fillna(0)  # Fill NaN with 0

# Display pivoted data
category_pivot.head()

category,date,Beauty,Books,Clothing,Electronics,Furniture,Groceries,Sports,Toys
0,2022-05-09,3211.85,11719.79,8991.86,12973.94,7052.30,10310.95,12460.70,9731.78
1,2022-05-10,9319.84,2351.34,5613.34,6798.78,1798.18,6152.45,8667.89,8931.23
2,2022-05-11,2880.05,5321.75,5921.33,7691.27,11678.49,11025.72,14755.56,4740.08
3,2022-05-12,12201.02,6192.31,4919.63,4892.80,10826.48,14126.95,11175.58,5048.10
4,2022-05-13,13638.68,8817.97,10661.11,12779.46,5663.99,9787.65,11295.66,6615.17


In [47]:
# Recompute daily context in case previous version was date-level only
daily_sales = sales_df.groupby('date').agg(
    total_sales=('revenue', 'sum'),
    avg_transaction=('revenue', 'mean'),
    transaction_count=('revenue', 'count'),
    unique_categories=('category', 'nunique')
).reset_index()

daily_sales.head()

,date,total_sales,avg_transaction,transaction_count,unique_categories
0,2022-05-09,76453.17,3185.548750,24,8
1,2022-05-10,49633.05,2068.043750,24,8
2,2022-05-11,64014.25,2667.260417,24,8
3,2022-05-12,69382.87,2890.952917,24,8
4,2022-05-13,79259.69,3302.487083,24,8


In [48]:
# Merge category features with daily sales
features_df = pd.merge(daily_sales, category_pivot, on='date', how='left')
features_df = features_df.fillna(0)  # Fill any NaN values

# Display final feature dataframe
features_df.head()

,date,total_sales,avg_transaction,transaction_count,unique_categories,Beauty,Books,Clothing,Electronics,Furniture,Groceries,Sports,Toys
0,2022-05-09,76453.17,3185.548750,24,8,3211.85,11719.79,8991.86,12973.94,7052.30,10310.95,12460.70,9731.78
1,2022-05-10,49633.05,2068.043750,24,8,9319.84,2351.34,5613.34,6798.78,1798.18,6152.45,8667.89,8931.23
2,2022-05-11,64014.25,2667.260417,24,8,2880.05,5321.75,5921.33,7691.27,11678.49,11025.72,14755.56,4740.08
3,2022-05-12,69382.87,2890.952917,24,8,12201.02,6192.31,4919.63,4892.80,10826.48,14126.95,11175.58,5048.10
4,2022-05-13,79259.69,3302.487083,24,8,13638.68,8817.97,10661.11,12779.46,5663.99,9787.65,11295.66,6615.17


## Feature Selection and Preparation
In this section, we finalize our feature set for model training. We handle missing values by dropping rows with NaN values that might have been introduced during the creation of lag and rolling features. We also ensure that all necessary temporal features are present in our final feature dataframe. This step is critical for preparing clean, consistent data that will yield reliable model performance. The final feature set combines all the engineered features from previous sections into a comprehensive dataset ready for model training.


In [49]:
# Drop rows with NaN values (from lag/rolling features)
features_df = features_df.dropna()

# Extract date components for the feature dataframe
features_df['year'] = features_df['date'].dt.year
features_df['month'] = features_df['date'].dt.month
features_df['day_of_week'] = features_df['date'].dt.dayofweek
features_df['is_weekend'] = features_df['day_of_week'].isin([5, 6]).astype(int)

# Display final feature set
features_df.head()

,date,total_sales,avg_transaction,transaction_count,unique_categories,Beauty,Books,Clothing,Electronics,Furniture,Groceries,Sports,Toys,year,month,day_of_week,is_weekend
0,2022-05-09,76453.17,3185.548750,24,8,3211.85,11719.79,8991.86,12973.94,7052.30,10310.95,12460.70,9731.78,2022,5,0,0
1,2022-05-10,49633.05,2068.043750,24,8,9319.84,2351.34,5613.34,6798.78,1798.18,6152.45,8667.89,8931.23,2022,5,1,0
2,2022-05-11,64014.25,2667.260417,24,8,2880.05,5321.75,5921.33,7691.27,11678.49,11025.72,14755.56,4740.08,2022,5,2,0
3,2022-05-12,69382.87,2890.952917,24,8,12201.02,6192.31,4919.63,4892.80,10826.48,14126.95,11175.58,5048.10,2022,5,3,0
4,2022-05-13,79259.69,3302.487083,24,8,13638.68,8817.97,10661.11,12779.46,5663.99,9787.65,11295.66,6615.17,2022,5,4,0


In [50]:
# Save the engineered features
features_df.to_csv('../data/processed/sales_engineered_features.csv', index=False)
print(f"Saved engineered features with shape: {features_df.shape}")

Saved engineered features with shape: (1096, 17)


## Conclusion

We've created a comprehensive set of features for our sales forecasting model, including:
- Time-based features (year, month, day, day of week, etc.)
- Lagged features (previous day, week, month)
- Rolling window statistics (7-day and 30-day means and standard deviations)
- Category-specific sales amounts

These features will be used in the next notebook for model training.
